In [16]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('/spam.csv')
print(data.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [18]:
%pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 262.6 kB/s eta 0:00:06
   - -------------------------------------- 0.0/1.5 MB 245.8 kB/s eta 0:00:06
   - -------------------------------------- 0.1/1.5 MB 328.2 kB/s eta 0:00:05
   - -------------------------------------- 0.1/1.5 MB 328.2 kB/s eta 0:00:05
   - -------------------------------------- 0.1/1.5 MB 245.8 kB/s eta 0:00:06
   - -------------------------------------- 0.1/1.5 MB 245.8 kB/s eta 0:00:06
   - -------------------------------------- 0.1/1.5 MB 245.8 kB/s eta 0:00:06
   ---- --------------------------------


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import nltk
from nltk.stem import WordNetLemmatizer

import re

nltk.download('punkt')
nltk.download('punkt_tab')

lemmatizer = WordNetLemmatizer()


def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Удаление спецсимволов
    text = text.lower()  # Приведение к нижнему регистру
    words = nltk.word_tokenize(text)  # Токенизация
    return ' '.join([lemmatizer.lemmatize(word) for word in words])
    
#print(preprocess_text('Downloading package wordnet to'))
data['processed_message'] = data['v2'].apply(lambda x: preprocess_text(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Дом\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Дом\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [29]:
print(data.head())

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4                                  processed_message  
0        NaN        NaN  go until jurong point crazy available only in ...  
1        NaN        NaN                            ok lar joking wif u oni  
2        NaN        NaN  free entry in 2 a wkly comp to win fa cup fina...  
3        NaN        NaN        u dun say so early hor u c already then say  
4        NaN        NaN  nah i don t think he go to usf he life around ...  


In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Bag of Words
bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(data['processed_message'])

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data['processed_message'])

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Подготовка данных
y = data['v1'].apply(lambda x: 1 if x == 'spam' else 0)
X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size=0.2, random_state=42)

# Обучение модели
model = LogisticRegression()
model.fit(X_train, y_train)

# Прогнозирование
y_pred = model.predict(X_test)

# Оценка модели
print('Bag of Words'+classification_report(y_test, y_pred))

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Обучение модели
model2 = LogisticRegression()
model2.fit(X_train2, y_train2)

# Прогнозирование
y_pred2 = model2.predict(X_test2)

# Оценка модели
print('TF-IDF'+classification_report(y_test2, y_pred2))


Bag of Words              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.99      0.85      0.91       150

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

TF-IDF              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.70      0.82       150

    accuracy                           0.96      1115
   macro avg       0.98      0.85      0.90      1115
weighted avg       0.96      0.96      0.96      1115



In [34]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

# Оценка ROC-AUC
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print(f'Bag of Words ROC-AUC: {roc_auc}, F1-score: {f1}, Accuracy: {accuracy}')

roc_auc2 = roc_auc_score(y_test2, model2.predict_proba(X_test2)[:, 1])
f1_2 = f1_score(y_test2, y_pred2)
accuracy2 = accuracy_score(y_test2, y_pred2)

print(f'TF-IDF ROC-AUC: {roc_auc2}, F1-score: {f1_2}, Accuracy: {accuracy2}')

Bag of Words ROC-AUC: 0.9830949913644215, F1-score: 0.9136690647482014, Accuracy: 0.97847533632287
TF-IDF ROC-AUC: 0.9842625215889464, F1-score: 0.8235294117647058, Accuracy: 0.9596412556053812
